# Newsgroups

In order to practice and learn to work with text data, let's use the 20 Newsgroups archive.

Things we're going to do:
- get the data
- work with some feature extraction
- develop a model that can classify the Newsgroups

In [1]:
## import essentials and anticipated packages

import numpy as np

from sklearn.datasets import fetch_20newsgroups

from sklearn.pipeline import pipeline
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer, CountVectorizer


from sklearn.naive_bayes import MultinomialNB

ModuleNotFoundError: No module named 'numpy'